In [1]:
"""
# 전처리 배분

1(성준) - 0~9,   40~49, 80~89, 120
       - 
2(장호) - 10~19, 50~59, 90~99, 121
3(두영) - 20~29, 60~69, 100~109, 122
4(혜빈) - 30~39, 70~79, 110~119, 123
5(준혁) - bureau.csv, bureau_balance.csv
"""

"""
# 데이터 목록
application_test.csv
application_train.csv
POS_CASH_balance.csv
bureau.csv
bureau_balance.csv
credit_card_balance.csv
installments_payments.csv
previous_application.csv
sample_submission.csv
"""

'\n# 데이터 목록\napplication_test.csv\napplication_train.csv\nPOS_CASH_balance.csv\nbureau.csv\nbureau_balance.csv\ncredit_card_balance.csv\ninstallments_payments.csv\nprevious_application.csv\nsample_submission.csv\n'

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
import user_functions as uf

In [3]:
# Load file
train_datasets = pd.read_csv(uf.get_f_path()+"application_train.csv")
test_datasets = pd.read_csv(uf.get_f_path()+"application_test.csv")

# Label 만들기
train_target_ds = train_datasets["TARGET"]

# TARGET feature 삭제
train_datasets.drop(columns="TARGET", inplace=True)

In [4]:
# info
print(train_datasets.info())
print(test_datasets.info())

# train은 307,511 행
# test는   48,744 행
# -----------------
# 총      356,255 행

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 121 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(40), object(16)
memory usage: 283.9+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Columns: 121 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(40), object(16)
memory usage: 45.0+ MB
None


In [5]:
# train datasets과 test datasets을 구분하는 "train/test" feature 생성
train_datasets["train/test"] = "train"
test_datasets["train/test"] = "test"

# 합치기
train_test_datasets = pd.concat([train_datasets, test_datasets], ignore_index=True)

# 356,255 행인지 확인
train_test_datasets.info()

# index 재설정
# print(train_datasets.index)
# print(test_datasets.index)
train_test_datasets.reset_index(drop=True, inplace=True)
# print(train_test_datasets.index)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356255 entries, 0 to 356254
Columns: 122 entries, SK_ID_CURR to train/test
dtypes: float64(65), int64(40), object(17)
memory usage: 331.6+ MB


In [6]:
# 전처리 데이터
prepro_ds = train_test_datasets.copy()

# 데이터 전처리 진행 및 특이사항
---

## "TARGET"
- 연체자가 적은 편향된 데이터
- value_counts()
- 1=연체
> ```python
> 0    282686
> 1     24825
> ```

<hr><br>

## "CODE_GENDER"
- 'M', 'F', 'XNA'
- 'XNA'는 결측치라기 보다는 간성 등으로 파악됨. 따라서 그대로 놔둠
- []숫자로 대치
- 성별
- value_counts()
> ```python
> F      202448
> M      105059
> XNA         4
> ```

<hr><br>

## "FLAG_OWN_CAR", "FLAG_OWN_REALTY"
- 'Y', 'N'
- []숫자로 대치

<hr><br>

## "CNT_CHILDREN"
- [V] 5명 이상인 값을 5 이상으로 처리
- []숫자로 대치
- value_counts()
> ```python
> 0     215371
> 1      61119
> 2      26749
> 3       3717
> 4        429
> 5         84
> 6         21
> 7          7
> 14         3
> 8          2
> 9          2
> 12         2
> 10         2
> 19         2
> 11         1
> ```

<hr><br>

## "AMT_INCOME_TOTAL"
- 117000000 값은 이상치로 판단됨. 대출액 대비 소득 수준을 볼 때 fatfinger로 판단됨
- [V] 117000000 / 10으로 대체함
- <span style="color:yellow">TODO: 편차가 크다고 판단되나, 우선 그대로 진행함</span>


<hr><br>

## "AMT_CREDIT"
- 큰 이상 없다고 판단됨
- <span style="color:yellow">TODO: 편차가 크다고 판단되나, 우선 그대로 진행함</span>

<hr><br>

## "AMT_ANNUITY", "AMT_GOODS_PRICE"
- 결측치를 대출금액을 기준으로 가장 유사한 값으로 대체함
- [V] 결측치 값을 `imput_other_f_mean` 함수를 활용해 대체함

<hr><br>

## "ORGANIZATION_TYPE"
- 무직자를 XNA로 처리함. 문제 없음
```python
prepro_ds.loc[prepro_ds["ORGANIZATION_TYPE"] == "XNA", ("NAME_INCOME_TYPE", "ORGANIZATION_TYPE")].value_counts()
```
> ```python
> NAME_INCOME_TYPE  ORGANIZATION_TYPE
> Pensioner         XNA                  55352
> Unemployed        XNA                     22
> ```

<hr><br>

## "EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"
- 외부 자료 출처에 대한 정규화된 점수
- [V] NAN 값이 많으나, 외부 자료에 대한 신뢰성(가중치)를 의미하므로 0으로 처리함

<hr><br>

## 기타 1
#### "APARTMENTS_AVG" 
#### "BASEMENTAREA_AVG" 
#### "YEARS_BEGINEXPLUATATION_AVG" 
#### "YEARS_BUILD_AVG" 
#### "COMMONAREA_AVG"
#### "ELEVATORS_AVG"
- 정규화된 자료
- 아파트 평균, 지하 면적 평균, 개발 시작연도 평균, 건물 연령 평균, 공용 공간 평균, 엘리베이터 수 평균
- [V] NAN 값이 많으나, 해당 정보가 없는 데이터라고 판단하여 0으로 처리함

## "ENTRANCES_AVG"
- 정규화된 자료
- 출입구 수의 평균을 의미함
- 0도 361개 존재함
- 결측값이 상당히 많음 178,407
- [V] <span style="color:yellow">중앙값으로 대체함</span>

<hr><br>

## "FONDKAPREMONT_MODE"
- 뭘 뜻하는 feature인지 알 수 없음
- [V] 결측치를 'not specified'로 대체함.

<hr><br>

## "HOUSETYPE_MODE"
- 결측값 154297
- [V] HOUSE에 거주하지 않는다고 판단하고 XNA로 처리함
- 나머지 값 분포
```python
block of flats      150503
specific housing      1499
terraced house        1212
```
<!-- - <span style="color:red">TODO: 처리 여부 결정</span> -->

<hr><br>

## 기타 2
#### "FLOORSMIN_MEDI"
#### "LANDAREA_MEDI"
#### "LIVINGAPARTMENTS_MEDI"
#### "LIVINGAREA_MEDI"
#### "NONLIVINGAPARTMENTS_MEDI"
#### "NONLIVINGAREA_MEDI"
#### "TOTALAREA_MODE"

- [V] NAN 값이 많으나, 해당 정보가 없는 데이터라고 판단하여 0으로 처리함

<hr><br>

## "WALLSMATERIAL_MODE"

- na 180234개로 상당히 많음.
- 따로 구분하기 어렵다고 판단하여 others로 처리함
```python
Panel           77309
Stone, brick    75249
Block           10681
Wooden           6156
Mixed            2649
Monolithic       2068
Others           1909
```

<hr><br>

## "AMT_REQ_CREDIT_BUREAU_QRT"
- 대출 신청 90일~30일 전 고객에 대한 CB 문의 건수
- unique = [  0.  nan   1.   2.   4.   3.   8.   5.   6.   7. 261.  19.]
- 261과 19는 다른 데이터와 비교했을 때 비정상적임. 가장 큰 값인 8로 대체함.


In [7]:
# 자녀가 5명 이상인 값을 5 이상으로 처리 후 숫자로 변환
prepro_ds.loc[prepro_ds["CNT_CHILDREN"] >= 5, "CNT_CHILDREN"] = "5 or more"

# 이상치 117000000에서 0을 하나 제거함
prepro_ds.loc[prepro_ds["AMT_INCOME_TOTAL"] > 1e+08, "AMT_INCOME_TOTAL"] = prepro_ds.loc[prepro_ds["AMT_INCOME_TOTAL"] > 1e+08, "AMT_INCOME_TOTAL"].map(lambda x: x / 10)

# "AMT_ANNUITY"를 "AMT_CREDIT"기준으로 가장 유사한 값을 통해 대체함
prepro_ds["AMT_ANNUITY"] = \
    uf.imput_other_f_mean(prepro_ds, "AMT_ANNUITY", "AMT_CREDIT")

# "AMT_GOODS_PRICE"를 "AMT_CREDIT"기준으로 가장 유사한 값을 통해 대체함
prepro_ds["AMT_GOODS_PRICE"] = \
    uf.imput_other_f_mean(prepro_ds, "AMT_GOODS_PRICE", "AMT_CREDIT")


# "EXT_SOURCE_1"의 결측치를 0으로 대체함
# 외부 자료의 신뢰성을 의미하는 자료이므로 외부 자료가 없는 것을 의미한다고 판단됨
prepro_ds["EXT_SOURCE_1"].fillna(0, inplace=True)
prepro_ds["EXT_SOURCE_2"].fillna(0, inplace=True)
prepro_ds["EXT_SOURCE_3"].fillna(0, inplace=True)
prepro_ds["FONDKAPREMONT_MODE"].fillna("not specified", inplace=True)

# 결측치를 0으로 대체함
zero_imputation_lists= ["FLOORSMIN_MEDI",
                        "LANDAREA_MEDI",
                        "LIVINGAPARTMENTS_MEDI",
                        "LIVINGAREA_MEDI",
                        "NONLIVINGAPARTMENTS_MEDI",
                        "NONLIVINGAREA_MEDI",
                        "TOTALAREA_MODE",
                        "APARTMENTS_AVG",
                        "BASEMENTAREA_AVG",
                        "YEARS_BEGINEXPLUATATION_AVG",
                        "YEARS_BUILD_AVG",
                        "COMMONAREA_AVG",
                        "ELEVATORS_AVG"]
for feature in zero_imputation_lists:
    prepro_ds.loc[:, feature].fillna(0, inplace=True)

# 중앙값으로 대체함
imput_mean = SimpleImputer(strategy="median")
prepro_ds["ENTRANCES_AVG"] = imput_mean.fit_transform(prepro_ds[["ENTRANCES_AVG"]])

# "WALLSMATERIAL_MODE" "Others"로 대체
prepro_ds["WALLSMATERIAL_MODE"].fillna("Others", inplace=True)

# 이상치를 정상값 범위 중 가장 큰 8로 대체함
prepro_ds["AMT_REQ_CREDIT_BUREAU_QRT"].fillna(8, inplace=True)

# "HOUSETYPE_MODE" 결측치 "XNA"로 대체 ∵ HOUSE외 주거 시설에 거주한다고 판단.
prepro_ds["HOUSETYPE_MODE"].fillna("XNA", inplace=True)


In [10]:
uf.check_missing_value(prepro_ds)

 10        NAME_TYPE_SUITE              2,203
 20          OWN_CAR_AGE              235,241
 27        OCCUPATION_TYPE            111,996
 28        CNT_FAM_MEMBERS                  2
 50         FLOORSMAX_AVG             176,341
 51         FLOORSMIN_AVG             241,108
 52         LANDAREA_AVG              210,844
 53     LIVINGAPARTMENTS_AVG          242,979
 54        LIVINGAREA_AVG             177,902
 55    NONLIVINGAPARTMENTS_AVG        246,861
 56       NONLIVINGAREA_AVG           195,766
 57        APARTMENTS_MODE            179,948
 58       BASEMENTAREA_MODE           207,584
 59 YEARS_BEGINEXPLUATATION_MODE      172,863
 60       YEARS_BUILD_MODE            236,306
 61        COMMONAREA_MODE            248,360
 62        ELEVATORS_MODE             189,080
 63        ENTRANCES_MODE             178,407
 64        FLOORSMAX_MODE             176,341
 65        FLOORSMIN_MODE             241,108
 66         LANDAREA_MODE             210,844
 67     LIVINGAPARTMENTS_MODE     

---
---

# credit_card_balance.csv
---

In [11]:
# load datasets
credit_card_ds = pd.read_csv(uf.get_f_path()+"credit_card_balance.csv")

In [12]:
credit_card_ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3840312 entries, 0 to 3840311
Data columns (total 23 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   SK_ID_PREV                  int64  
 1   SK_ID_CURR                  int64  
 2   MONTHS_BALANCE              int64  
 3   AMT_BALANCE                 float64
 4   AMT_CREDIT_LIMIT_ACTUAL     int64  
 5   AMT_DRAWINGS_ATM_CURRENT    float64
 6   AMT_DRAWINGS_CURRENT        float64
 7   AMT_DRAWINGS_OTHER_CURRENT  float64
 8   AMT_DRAWINGS_POS_CURRENT    float64
 9   AMT_INST_MIN_REGULARITY     float64
 10  AMT_PAYMENT_CURRENT         float64
 11  AMT_PAYMENT_TOTAL_CURRENT   float64
 12  AMT_RECEIVABLE_PRINCIPAL    float64
 13  AMT_RECIVABLE               float64
 14  AMT_TOTAL_RECEIVABLE        float64
 15  CNT_DRAWINGS_ATM_CURRENT    float64
 16  CNT_DRAWINGS_CURRENT        int64  
 17  CNT_DRAWINGS_OTHER_CURRENT  float64
 18  CNT_DRAWINGS_POS_CURRENT    float64
 19  CNT_INSTALMENT_MATURE

## 1. 정규화


## 2. 결측치 to 0
```python
["AMT_DRAWINGS_ATM_CURRENT",
"AMT_DRAWINGS_OTHER_CURRENT"
"AMT_DRAWINGS_POS_CURRENT",
"AMT_INST_MIN_REGULARITY",
"AMT_PAYMENT_CURRENT",
"CNT_DRAWINGS_ATM_CURRENT",
"CNT_DRAWINGS_OTHER_CURRENT"
"CNT_DRAWINGS_POS_CURRENT",
"CNT_INSTALMENT_MATURE_CUM",]
```

In [13]:
# "MONTHS_BALANCE" = 대출 신청일 - 신용카드 잔액 갱신일
# 정규화 필요함
# uf.check_feature(credit_card_ds, "MONTHS_BALANCE")

# "AMT_BALANCE" = 총 신용카드 결제 예정금액
# 정규화 필요
# uf.check_feature(credit_card_ds, "AMT_BALANCE")

# "AMT_CREDIT_LIMIT_ACTUAL" = 신용카드 한도
# uf.check_feature(credit_card_ds, "AMT_CREDIT_LIMIT_ACTUAL")

# "AMT_DRAWINGS_CURRENT" = 인출액
# uf.check_feature(credit_card_ds, "AMT_DRAWINGS_CURRENT")

# "AMT_PAYMENT_TOTAL_CURRENT" = 신용카드 총 사용액
# uf.check_feature(credit_card_ds, "AMT_PAYMENT_TOTAL_CURRENT")

# "AMT_RECEIVABLE_PRINCIPAL" = 미수(원)금
# uf.check_feature(credit_card_ds, "AMT_RECEIVABLE_PRINCIPAL")

# "AMT_RECIVABLE" = 미수금
# uf.check_feature(credit_card_ds, "AMT_RECIVABLE")

# "AMT_TOTAL_RECEIVABLE" = 미수금 총액
# uf.check_feature(credit_card_ds, "AMT_TOTAL_RECEIVABLE")

# "CNT_DRAWINGS_CURRENT" = 인출 횟수
# uf.check_feature(credit_card_ds, "CNT_DRAWINGS_CURRENT")
#####################################################################
                            # 결측치 0으로 처리함
#####################################################################
# "AMT_DRAWINGS_ATM_CURRENT" = ATM 인출액
# uf.check_feature(credit_card_ds, "AMT_DRAWINGS_ATM_CURRENT")

# "AMT_DRAWINGS_OTHER_CURRENT" = 기타 인출액
# uf.check_feature(credit_card_ds, "AMT_DRAWINGS_OTHER_CURRENT")

# "AMT_DRAWINGS_POS_CURRENT" = 인출 또는 구매 금액
# uf.check_feature(credit_card_ds, "AMT_DRAWINGS_POS_CURRENT")

# "AMT_INST_MIN_REGULARITY" = 최소 할부금
# uf.check_feature(credit_card_ds, "AMT_INST_MIN_REGULARITY")

# "AMT_PAYMENT_CURRENT" = 신용카드 사용액
# uf.check_feature(credit_card_ds, "AMT_PAYMENT_CURRENT")
# "CNT_DRAWINGS_ATM_CURRENT" = ATM 인출 횟수
# uf.check_feature(credit_card_ds, "CNT_DRAWINGS_ATM_CURRENT")

# "CNT_DRAWINGS_OTHER_CURRENT" = 기타 인출 횟수
# uf.check_feature(credit_card_ds, "CNT_DRAWINGS_OTHER_CURRENT")

# "CNT_DRAWINGS_POS_CURRENT" = 제품 구매를 위한 인출 횟수
# uf.check_feature(credit_card_ds, "CNT_DRAWINGS_POS_CURRENT")

# "CNT_INSTALMENT_MATURE_CUM" = 할부금 지불 횟수
# uf.check_feature(credit_card_ds, "CNT_INSTALMENT_MATURE_CUM")
#####################################################################
zero_imputation_lists2 = ["AMT_DRAWINGS_ATM_CURRENT",
                          "AMT_DRAWINGS_OTHER_CURRENT",
                          "AMT_DRAWINGS_POS_CURRENT",
                          "AMT_INST_MIN_REGULARITY",
                          "AMT_PAYMENT_CURRENT",
                          "CNT_DRAWINGS_ATM_CURRENT",
                          "CNT_DRAWINGS_OTHER_CURRENT",
                          "CNT_DRAWINGS_POS_CURRENT",
                          "CNT_INSTALMENT_MATURE_CUM",
                         ]

for feature in zero_imputation_lists2:
    credit_card_ds.loc[:, feature].fillna(0, inplace=True)

#####################################################################


# "NAME_CONTRACT_STATUS" = 계약 상태
# uf.check_feature(credit_card_ds, "NAME_CONTRACT_STATUS")

# "SK_DPD" = 연체 경과일
# uf.check_feature(credit_card_ds, "SK_DPD")

# "SK_DPD_DEF" = 연체 경과일(소액 연체 제외)
# uf.check_feature(credit_card_ds, "SK_DPD_DEF")

In [14]:
def scaling_and_encoding(datasets, scaling=True, encoding=True):
    # dtype이 object인 features 리스트 생성
    categorical_feature = \
        datasets.dtypes[datasets.dtypes == "object"].index.to_list()

    # dtype이 object가 아닌 numerical features 리스트 생성
    numerical_feature = \
        datasets.dtypes[datasets.dtypes != "object"].index.to_list()
    
    ##########################################################
    # 숫자형 features를 RobustScaler()적용
    ##########################################################
    if scaling == True:
        for feature in numerical_feature:
            rbscaler = RobustScaler()
            
            # RobustScaler로 datasets 변환
            # numpy.ndarray 반환됨
            temp_rbscaler = rbscaler.fit_transform(pd.DataFrame(datasets[feature]))

            # 데이터 수정
            datasets[feature] = temp_rbscaler
            # numpy.ndarray에서 pandas.DataFrame으로 변환할 경우
            # pd.DataFrame(c_card_rbscaler, columns=temp_c_card.columns)   
    ##########################################################

    ##########################################################
    # 문자형 features를 LabelEncoder()적용
    ##########################################################
    if encoding == True:
        for feature in categorical_feature:
            # LabelEncoder object 생성
            l_encod = LabelEncoder()

            # fit을 통해 인코딩 수행
            # numpy.ndarray 반환됨
            temp_encod = l_encod.fit_transform(datasets[feature].ravel())
            
            # 데이터 수정
            datasets[feature] = temp_encod

    return datasets

In [15]:
# LabelEncoder() 적용
confirmed_c_card = uf.scaling_and_encoding(credit_card_ds.iloc[:, 2:], scaling=False, encoding=True)